- **[実験](#実験)**
    - [準備](#準備)
    - [データ量](#データ量)
    - [バッチサイズ](#バッチサイズ)
    - [活性化関数の種類](#活性化関数の種類)
    - [最適化関数](#最適化関数)
    - [bound](#bound)
    - [更新回数](#更新回数)
    - [調査ターン](#調査ターン)
    - [hook_function](#hook_function)
- **[組み合わせ実験](#組み合わせ実験)**
- **[組み合わせ実験2](#組み合わせ実験2)**
- **[学習済みモデル作成](#学習済みモデル作成)**

# 実験

## 準備

In [ ]:
import matplotlib.pyplot as plt

from deep_learn import *

## データ量

In [4]:
run = deep_learn()

num_arr = [1, 5, 10]
run.fig, run.ax = plt.subplots(ncols=2, nrows=len(num_arr), figsize=(10, len(num_arr)*5))
plt.subplots_adjust(wspace=None, hspace=0.2)
i = 1
MSE_test = []
MAE_test = []

for num in num_arr:
    print("\r%d/%d" % (i, len(num_arr)), end="")
    run.num = num
    run.fig_name1 = "MSE of each epoch (data quantity is %d)" % num
    run.fig_name2 = "MAE of each epoch (data quantity is %d)" % num
    run.set_data()
    run.fit()
    run.plot(i)
    test_error = run.cal_test_error()
    MSE_test.append(test_error[0])
    MAE_test.append(test_error[1])
    i += 1

run.fig.savefig("fig/data_quantity.png")
run.fig.clf()

3/3

<Figure size 720x1080 with 0 Axes>

In [5]:
print("number: ", num_arr)
print("MSE: ", MSE_test)
print("MAE: ", MAE_test)

number:  [1, 5, 10]
MSE:  [variable(152.81169), variable(372.01184), variable(643.15546)]
MAE:  [variable(8.03122), variable(15.009193), variable(20.94951)]


## バッチサイズ

In [21]:
run = deep_learn()

batch_arr = [100, 200, 300, 500, 700, 1000, 1500]
run.fig, run.ax = plt.subplots(ncols=2, nrows=len(batch_arr), figsize=(10, len(batch_arr)*5))
plt.subplots_adjust(wspace=None, hspace=0.2)
i = 1
MSE_test = []
MAE_test = []

run.set_data()

for batch_size in batch_arr:
    print("\r%d/%d" % (i, len(batch_arr)), end="")
    run.batch_size = batch_size
    run.fig_name1 = "MSE of each epoch (batch size is %d)" % batch_size
    run.fig_name2 = "MAE of each epoch (batch size is %d)" % batch_size
    run.fit()
    run.plot(i)
    test_error = run.cal_test_error()
    MSE_test.append(test_error[0])
    MAE_test.append(test_error[1])
    i += 1

run.fig.savefig("fig/batch_size.png")
run.fig.clf()

7/7

<Figure size 720x2520 with 0 Axes>

In [22]:
print("batch size: ", batch_arr)
print("MSE: ", MSE_test)
print("MAE: ", MAE_test)

batch size:  [100, 200, 300, 500, 700, 1000, 1500]
MSE:  [variable(162.9739), variable(152.05502), variable(153.27628), variable(162.2756), variable(145.58446), variable(204.26527), variable(167.85017)]
MAE:  [variable(8.527661), variable(8.186351), variable(8.13009), variable(8.399555), variable(7.9057674), variable(9.92684), variable(8.66958)]


## 活性化関数の種類

In [13]:
import chainer.functions as F

import Net

run = deep_learn()

f_arr = [
    F.clipped_relu,
    F.elu,
    F.leaky_relu,
    # F.log_softmax,  overflow
    F.rrelu,
    F.relu,
    F.sigmoid,
    F.softmax,
    # F.softplus,    overflow
    F.tanh
]
f_name = [
    "clipped_relu",
    "elu",
    "leaky_relu",
    # "log_softmax",
    "rrelu",
    "relu",
    "sigmoid",
    "softmax",
    # "softplus",
    "tanh"
]
run.fig, run.ax = plt.subplots(ncols=2, nrows=len(f_arr), figsize=(10, len(f_arr)*5))
plt.subplots_adjust(wspace=None, hspace=0.2)
i = 1
MSE_test = []
MAE_test = []

run.set_data()

for function in f_arr:
    print("\r%d/%d" % (i, len(f_arr)), end="")
    Net.func = function
    run.Net = Net.Net
    run.fig_name1 = "MSE of each epoch (num of function is %s)" % f_name[i-1]
    run.fig_name2 = "MAE of each epoch (num of function is %s)" % f_name[i-1]
    run.fit()
    run.plot(i)
    test_error = run.cal_test_error()
    MSE_test.append(test_error[0])
    MAE_test.append(test_error[1])
    i += 1

run.fig.savefig("fig/function")
run.fig.clf()

8/8

<Figure size 720x2880 with 0 Axes>

In [14]:
print("function: ", f_name)
print("MSE: ", MSE_test)
print("MAE: ", MAE_test)

function:  ['clipped_relu', 'elu', 'leaky_relu', 'rrelu', 'relu', 'sigmoid', 'softmax', 'tanh']
MSE:  [variable(234.85422), variable(435.0493), variable(1528.1886), variable(1051.0135), variable(451.09824), variable(294.6485), variable(571.89453), variable(171.02132)]
MAE:  [variable(10.507885), variable(16.28336), variable(33.503345), variable(28.046625), variable(15.27029), variable(13.055871), variable(20.325909), variable(9.534906)]


## 最適化関数

In [8]:
# https://qiita.com/ZoneTsuyoshi/items/8ef6fa1e154d176e25b8

from chainer import optimizers

opt_arr = [
    optimizers.SGD(),
    optimizers.MomentumSGD(),
    optimizers.AdaGrad(),
    optimizers.RMSprop(),
    optimizers.AdaDelta(),
    optimizers.Adam(),
    optimizers.RMSpropGraves(),
    optimizers.SMORMS3(),
    optimizers.AMSGrad(),
    optimizers.AdaBound(),
    optimizers.AMSBound()
]

opt_name = [
    "SGD",
    "MomentumSGD",
    "AdaGrad",
    "RmSprop",
    "AdaDelta",
    "Adam",
    "RMSpropGraves",
    "SMORMS3",
    "AMSGrad",
    "AdaBound",
    "AMSBound"
]

run = deep_learn()
run.fig, run.ax = plt.subplots(ncols=2, nrows=len(opt_arr), figsize=(10, len(opt_arr)*5))
plt.subplots_adjust(wspace=None, hspace=0.2)
i = 1
MSE_test = []
MAE_test = []

run.set_data()

for opt in opt_arr:
    print("\r%d/%d" % (i, len(opt_arr)), end="")
    run.optimizer = opt
    run.fig_name1 = "MSE of each epoch (optimizer is %s)" % opt_name[i - 1]
    run.fig_name2 = "MAE of each epoch (optimizer is %s)" % opt_name[i - 1]
    run.fit()
    run.plot(i)
    test_error = run.cal_test_error()
    MSE_test.append(test_error[0])
    MAE_test.append(test_error[1])
    i += 1

run.fig.savefig("fig/optmizer")
run.fig.clf()

11/11

<Figure size 720x3960 with 0 Axes>

In [9]:
print("opt name: ", opt_name)
print("MSE: ", MSE_test)
print("MAE: ", MAE_test)

opt name:  ['SGD', 'MomentumSGD', 'AdaGrad', 'RmSprop', 'AdaDelta', 'Adam', 'RMSpropGraves', 'SMORMS3', 'AMSGrad', 'AdaBound', 'AMSBound']
MSE:  [variable(220.08337), variable(576.7117), variable(251.20909), variable(569.64264), variable(184.31363), variable(162.6182), variable(156.20638), variable(161.93105), variable(169.1945), variable(150.75018), variable(139.02298)]
MAE:  [variable(10.743682), variable(20.432753), variable(12.183864), variable(20.282295), variable(9.238301), variable(9.405695), variable(8.508823), variable(9.181299), variable(8.943705), variable(8.068955), variable(7.9657555)]


## bound

In [2]:
run = deep_learn()

bound_arr = [1, 2, 3, 4, 5, 10]
run.fig, run.ax = plt.subplots(ncols=2, nrows=len(bound_arr), figsize=(10, len(bound_arr)*5))
plt.subplots_adjust(wspace=None, hspace=0.2)
i = 1
MSE_test = []
MAE_test = []

run.set_data()

for bound in bound_arr:
    print("\r%d/%d" % (i, len(bound_arr)), end="")
    run.bound = bound
    run.fig_name1 = "MSE of each epoch (bound is %d)" % bound
    run.fig_name2 = "MAE of each epoch (bound is %d)" % bound
    run.fit()
    run.plot(i)
    test_error = run.cal_test_error()
    MSE_test.append(test_error[0])
    MAE_test.append(test_error[1])
    i += 1

run.fig.savefig("fig/bound")
run.fig.clf()

6/6

<Figure size 720x2160 with 0 Axes>

In [3]:
print("bound :", bound_arr)
print("MSE: ", MSE_test)
print("MAE: ", MAE_test)

bound : [1, 2, 3, 4, 5, 10]
MSE:  [variable(208.39401), variable(195.04422), variable(211.6011), variable(189.95116), variable(244.27588), variable(212.02213)]
MAE:  [variable(10.187112), variable(9.844666), variable(10.383668), variable(10.060881), variable(11.47002), variable(10.53158)]


## 更新回数

In [15]:
run = deep_learn()

n_epoch_arr = [100, 1000]
run.fig, run.ax = plt.subplots(ncols=2, nrows=len(n_epoch_arr), figsize=(10, len(n_epoch_arr)*5))
plt.subplots_adjust(wspace=None, hspace=0.2)

run.set_data()
i = 1
MSE_test = []
MAE_test = []

for n_epoch in n_epoch_arr:
    run.n_epoch = n_epoch
    run.fig_name1 = "MSE of each epoch (n_epohc is %d)" % run.n_epoch
    run.fig_name2 = "MAE of each epoch (n_epoch is %d)" % run.n_epoch

    run.fit()
    run.plot(i)
    test_error = run.cal_test_error()
    MSE_test.append(test_error[0])
    MAE_test.append(test_error[1])
    i += 1

run.fig.savefig("fig/n_epoch")
run.fig.clf()

<Figure size 720x720 with 0 Axes>

In [16]:
print("n_epoch: ", n_epoch_arr)
print("MSE: ", MSE_test)
print("MAE: ", MAE_test)

n_epoch:  [100, 1000]
MSE:  [variable(152.18834), variable(144.9502)]
MAE:  [variable(7.6830935), variable(7.124355)]


## 調査ターン

In [10]:
import osero_learn as ol
import deep_learn as dl

run = dl.deep_learn()

cp_arr = [
    [i for i in range(1, 61)],
    [i for i in range(5, 61, 5)],
    [i for i in range(10, 61, 10)]
]
cp_name = [
    "every 1",
    "every 5",
    "every 10"
]
run.fig, run.ax = plt.subplots(ncols=2, nrows=len(cp_arr), figsize=(10, len(cp_arr)*5))
plt.subplots_adjust(wspace=None, hspace=0.2)
i = 1
MSE_test = []
MAE_test = []

for cp in cp_arr:
    print("\r%d/%d" % (i, len(cp_arr)), end="")
    run.osero = ol.learn(0, 0, check_point=cp, eva=[0, 0])
    run.set_data()
    run.fig_name1 = "MSE of each epoch (turn is %s)" % cp_name[i - 1]
    run.fig_name2 = "MAE of each epoch (turn is %s)" % cp_name[i - 1]
    run.fit()
    run.plot(i)
    test_error = run.cal_test_error()
    MSE_test.append(test_error[0])
    MAE_test.append(test_error[1])
    i += 1

run.fig.savefig("fig/check_point")
run.fig.clf()

3/3

<Figure size 720x1080 with 0 Axes>

In [11]:
print(cp_name)
print("MSE: ", MSE_test)
print("MAE: ", MAE_test)

['every 1', 'every 5', 'every 10']
MSE:  [variable(194.50024), variable(512.78925), variable(447.7704)]
MAE:  [variable(9.789661), variable(17.579159), variable(16.539795)]


## hook_function

In [13]:
import chainer.optimizer_hooks as hf

run = deep_learn()

hf_arr = [
    hf.WeightDecay(0.00001),
    hf.Lasso(0.00001),
    # hf.GradientClipping(1.0),
    hf.GradientHardClipping(-2, 2),
    hf.GradientNoise(0.3),
    hf.GradientLARS()
]
hf_name = [
    "WeightDecay",
    "Lasso",
    # "GradientClipping",
    "GradientHardClipping",
    "GradientNoise",
    "GradientLARS"
]
run.fig, run.ax = plt.subplots(ncols=2, nrows=len(hf_arr), figsize=(10, len(hf_arr)*5))
plt.subplots_adjust(wspace=None, hspace=0.2)
i = 1
MSE_test = []
MAE_test = []

run.set_data()

for hook_f in hf_arr:
    print("\r%d/%d" % (i, len(hf_arr)), end="")
    run.hook_f = hook_f
    run.fig_name1 = "MSE of each epoch (hook function is %s)" % hf_name[i - 1]
    run.fig_name2 = "MAE of each epoch (hook function is %s)" % hf_name[i - 1]
    run.fit()
    run.plot(i)
    test_error = run.cal_test_error()
    MSE_test.append(test_error[0])
    MAE_test.append(test_error[1])
    i += 1

run.fig.savefig("fig/hook_function")
run.fig.clf()

5/5

<Figure size 720x1800 with 0 Axes>

In [14]:
print(hf_name)
print("MSE: ", MSE_test)
print("MAE: ", MAE_test)

['WeightDecay', 'Lasso', 'GradientHardClipping', 'GradientNoise', 'GradientLARS']
MSE:  [variable(162.75676), variable(200.36641), variable(209.13129), variable(156.30049), variable(235.34283)]
MAE:  [variable(9.0745125), variable(10.085787), variable(9.9866905), variable(8.721768), variable(11.526583)]


# 組み合わせ実験

In [1]:
import chainer.functions as F
import chainer.optimizers as opt
import chainer.optimizer_hooks as hf
import pandas as pd

import Net
import deep_learn

In [2]:
num = 1
batch_size = 1000
f_arr = [
    F.sigmoid,
    F.tanh
]
f_name = [
    "sigmoid",
    "tanh"
]
opt_arr = [
    opt.AdaDelta(),
    opt.Adam(),
    opt.RMSpropGraves(),
    opt.SMORMS3(),
    opt.AMSGrad(),
    opt.AdaBound(),
    opt.AMSBound()
]
opt_name = [
    "AdaDelta",
    "Adam",
    "RMSpropGraves",
    "SMORMS3",
    "AMSGrad",
    "AdaBound",
    "AMSBound"
]
hf_arr = [
    hf.WeightDecay(0.00001),
    hf.Lasso(0.00001),
    hf.GradientHardClipping(-2, 2),
    hf.GradientNoise(0.3),
    hf.GradientLARS()
]
hf_name = [
    "WeightDecay",
    "Lasso",
    "GradientHardClipping",
    "GradientNoise",
    "GradientLARS"
]

In [3]:
data = {}

data["function"] = []
data["optimizer"] = []
data["hook_function"] = []
data["test_MSE"] = []
data["test_MAE"] = []

In [4]:
run = deep_learn.deep_learn()
run.num = num
run.batch_size = batch_size
run.set_data()
i, j, k = 0, 0, 0

for function in f_arr:
    Net.func = function
    run.Net = Net.Net
    j = 0
    for optimizer in opt_arr:
        run.optimizer = optimizer
        k = 0
        for hook_function in hf_arr:
            loading = "%d/%d" % (i+1, len(f_arr))\
                    + "#" * (j + 1)\
                    + "." * (k + 1)
            print("\r" + loading, end="")
            run.hook_f = hook_function
            run.fit()
            test_error = run.cal_test_error()
            data["function"].append(f_name[i])
            data["optimizer"].append(opt_name[j])
            data["hook_function"].append(hf_name[k])
            data["test_MSE"].append(float(test_error[0].array))
            data["test_MAE"].append(float(test_error[1].array))
            k += 1
        j += 1
    i += 1

2/2#######.....

In [5]:
data_df = pd.DataFrame(data)
data_df.to_csv("data.csv")

# 組み合わせ実験2

In [1]:
import chainer.functions as F
import chainer.optimizers as opt
import chainer.optimizer_hooks as hf
import pandas as pd

import Net
import deep_learn

In [4]:
num = 1
batch_arr = [200, 600, 1000]
func = F.tanh
opt_arr = [
    opt.AdaDelta(),
    opt.Adam(),
    opt.RMSpropGraves(),
    opt.SMORMS3(),
    opt.AMSGrad(),
    opt.AdaBound(),
    opt.AMSBound()
]
opt_name = [
    "AdaDelta",
    "Adam",
    "RMSpropGraves",
    "SMORMS3",
    "AMSGrad",
    "AdaBound",
    "AMSBound"
]
hf_arr = [
    hf.WeightDecay(0.00001),
    hf.Lasso(0.00001),
    hf.GradientHardClipping(-2, 2),
    hf.GradientLARS()
]
hf_name = [
    "WeightDecay",
    "Lasso",
    "GradientHardClipping",
    "GradientLARS"
]

In [6]:
data = {}

data["batch_size"] = []
data["optimizer"] = []
data["hook_function"] = []
data["test_MSE"] = []
data["test_MAE"] = []

In [7]:
run = deep_learn.deep_learn()
run.num = num
Net.func = func
run.Net = Net.Net
run.set_data()
i, j, k = 0, 0, 0

for batch_size in batch_arr:
    run.batch_size = batch_size
    j = 0
    for optimizer in opt_arr:
        run.optimizer = optimizer
        k = 0
        for hook_function in hf_arr:
            loading = "%d/%d" % (i+1, len(batch_arr))\
                    + "#" * (j + 1)\
                    + "." * (k + 1)
            print("\r" + loading, end="")
            run.hook_f = hook_function
            run.fit()
            test_error = run.cal_test_error()
            data["batch_size"].append(batch_size)
            data["optimizer"].append(opt_name[j])
            data["hook_function"].append(hf_name[k])
            data["test_MSE"].append(float(test_error[0].array))
            data["test_MAE"].append(float(test_error[1].array))
            k += 1
        j += 1
    i += 1

3/3#######....

In [8]:
data_df = pd.DataFrame(data)
data_df.to_csv("data2.csv")

# 学習済みモデル作成

In [12]:
import chainer.functions as F
from chainer.optimizers import AMSBound
from chainer.optimizer_hooks import Lasso
import matplotlib.pyplot as plt
from chainer.serializers import save_npz

import deep_learn
import Net

In [6]:
run = deep_learn.deep_learn()

run.num = 1
run.batch_size = 200
Net.func = F.tanh
run.Net = Net.Net
run.optimizer = AMSBound()
run.n_epoch = 150
run.hook_f = Lasso(0.00001)

In [7]:
run.set_data()
run.fit()

In [15]:
error = run.cal_test_error()
error

(variable(140.72816), variable(7.2066855))

In [10]:
fig = plt.figure(figsize=(10, 10))
plt.plot(run.results_train["MSE"], label="train")
plt.plot(run.results_valid["MSE"], label="valid")
plt.legend()
plt.xlabel("epoch number")
plt.ylabel("mean squared error")
plt.title("MSE of each epoch")
plt.savefig("fig/model_MSE")
plt.clf()
plt.close()

In [11]:
fig = plt.figure(figsize=(10, 10))
plt.plot(run.results_train["MAE"], label="train")
plt.plot(run.results_valid["MAE"], label="valid")
plt.legend()
plt.xlabel("epoch number")
plt.ylabel("mean absolute error")
plt.title("MAE of each epoch")
plt.savefig("fig/model_MAE")
plt.clf()
plt.close()

In [13]:
save_npz("model.net", run.net)